# Question 3

**IMPORTANT** 
<br><br>
Covid dataset is not consistent i.e., Given dataset store information in cummulative order but at some point it is not cummulative.<br><br>
For example total confirmed case on date(i) = 50 and date(i+1) = 40.<br><br>
To solve this problem I find all information for each day (info_date(i) = cum_info_date(i) - cum_info_date(i-1)). If difference is negative I assumed at that day no  new were cases found means storing 0.

Therefore summation of the info may not be equal to cummulative covid dataset due to it's inconsistency.

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
import json

Importing **state_to_code_dict.json** for using state_code and District_Key from vaccination dataset created in **Question 1**

In [2]:
f = open("Dataset/Additional Files/state_to_code_dict.json")
state_code = json.load(f)
state_code["Andaman and Nicobar Islands"] = "AN"

### Importing Covid Dataset

In [3]:
df_covid = pd.read_csv("Dataset/Input Files/districts.csv" )
df_covid = df_covid[df_covid.Date.between('2020-03-15', '2021-08-14')]
#List of unwanted district from covid dataset
unwanted_districts1 = set(["Unknown", "Airport Quarantine", "BSF Camp", "Evacuees", "Capital Complex", "Foreign Evacuees", "Italians", "Other Region",
                                         "Other State", "Others", "Railway Quarantine", "State Pool"])

#Droping unwanted list from the covid dataset
for dis in unwanted_districts1:
    df_covid = df_covid[df_covid.District != dis]
    
#Creating new columns for State_Code and District Key
df_covid["State_Code"] = ""
df_covid["District_Key"] = ""
for st in set(df_covid["State"]):
    df_covid.loc[df_covid["State"] == st, "State_Code"] = state_code[st]
    for dis in set(df_covid[df_covid["State"] == st]["District"]):
        df_covid.loc[((df_covid["State"] == st) & (df_covid["District"] == dis)), "District_Key"] = state_code[st] + str("_") + dis

#Putting only data upto '2021-08-14'

Removing unnecessary features

In [4]:
df_covid = df_covid.loc[:, ["Date", "District_Key", "State_Code", "Confirmed"]]

**unique_district** : Unique District from covid dataset
<br>
**unique_state** : Unique State from covid dataset

In [5]:
unique_district = set(df_covid["District_Key"])
unique_state = set(df_covid["State_Code"])

**date_range** is storing all the dates from 15 March 2020 to 14 August 2021

In [6]:
date_range = set([str(x)[:-9] for x in pd.date_range(start="2020-03-15", end="2021-08-14")])

#### Adding New Entry
Our Covid Dataset provides information from 26 April 2020 and for some district it has entry from latter dates.
<br>
In below cell we are adding new row for each and every district for which no record present in dataset.
<br>
date_diff :  it will store the dates for which no record is present


In [7]:
for dis in set(df_covid["District_Key"]):
    
    date_in_covid_file = list(df_covid[df_covid["District_Key"] == dis]["Date"])
        
    date_diff = []
   
    #Finding the date not present in covid dataset for district dis
    for d in date_range:
        if d not in date_in_covid_file:
            date_diff.append(d)
                
    n = len(date_diff)
    state_code = list(df_covid[df_covid["District_Key"] == dis]["State_Code"])[0]
    
    # Storing value for date which is not in dataset
    dist = {"Date" : date_diff,
           "Confirmed" : [0] * n,
            "State_Code" : [state_code] * n,
            "District_Key" : [dis] * n}
        
    df = pd.DataFrame(dist)
    df_covid = pd.concat([df_covid, df], ignore_index=True)
    df_covid.reset_index()
    
df_covid = df_covid.sort_values(by = ["Date", "District_Key"])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Since our dataset ahs cumullative reocrds we need to find exact data for that day
<br>
<br>
**modified_df_covid** : This DataFrame will stores all the record in non cummulative formate for each date in date_range and for each and every district

In [8]:
dic = {}

dic["Date"] = []
dic["Confirmed"] = []
dic["State_Code"] = []
dic["District_Key"] = []

modified_df_covid = pd.DataFrame(dic)

Bellow cell is used to fill **modified_df_covid** 

**data[i] = cummulative_data[i] - cummulative_data[i-1**

In [9]:
columns = df_covid.columns
index = list(columns).index("Confirmed")
for dis in set(df_covid["District_Key"]):
        
    distriict = np.array(df_covid[df_covid["District_Key"] == dis])
    l1 = []
    l1.append(distriict[0])
    for i in range(1, len(distriict)):
        l2 = []
        for j in  range(4):
            if j == index:
                
                l2.append(0 if (distriict[i][j] - distriict[i-1][j]) < 0 else distriict[i][j] - distriict[i-1][j])
                
            else:
                
                l2.append(distriict[i][j])
                
        l1.append(l2)
    df = pd.DataFrame(l1, columns = list(df_covid.columns))
    modified_df_covid = pd.concat([modified_df_covid, df], ignore_index=True)
    modified_df_covid.reset_index()
modified_df_covid.sort_values(by=["Date", "District_Key"], inplace=True)   

### District Weekly cases 

In [10]:
weekly_cases_list = []
weekly_cases_overlaped_list = []

for dis in set(modified_df_covid["District_Key"]):
    
    #initiating value of day and week id
    i = 0     #day starting from Sunday
    j = 4     #day starting from Thrusday
    time_id =  1     #Week id
    time_id_overlaped = 1
    
    #df will store just the dataframe only for single district to speed up
    df = np.array(modified_df_covid[modified_df_covid["District_Key"] == dis]["Confirmed"])
    
    while i < len(date_range):
        
        #Adding the confirmed cased for a week
        cases = np.sum(df[i : i + 7])
        weekly_cases_list.append([dis, time_id, cases])
        weekly_cases_overlaped_list.append([dis, time_id_overlaped, cases])
        
        #increasing the week Id by 1
        time_id = time_id + 1
        time_id_overlaped = time_id_overlaped + 1
        
        #Increasing the date by 7
        i = i + 7
        
        #Storing weekly cases starting from Thrusday
        cases = np.sum(df[j : j + 7])
        
        weekly_cases_overlaped_list.append([dis, time_id_overlaped, cases])
        
        time_id_overlaped = time_id_overlaped + 1

        j = j + 7
        
            
weekly_cases_df = pd.DataFrame(weekly_cases_list, columns=["districtid", "timeid", "cases"])
weekly_cases_df.sort_values(by=["districtid", "timeid"], inplace=True)

weekly_cases_overlap_df = pd.DataFrame(weekly_cases_overlaped_list, columns=["districtid", "timeid", "cases"])
weekly_cases_overlap_df.sort_values(by=["districtid", "timeid"], inplace=True)

### District Monthly Cases

**month_wise_date** : Store the starting and ending of each month from 15 march 2020 to 14 august 2021

In [11]:
month_wise_date = [["2020-03-15", "2020-04-14"], 
                  ["2020-04-15", "2020-05-14"], 
                   ["2020-05-15", "2020-06-14"],
                  ["2020-06-15", "2020-07-14"],
                  ["2020-07-15", "2020-08-14"],
                  ["2020-08-15", "2020-09-14"],
                  ["2020-09-15", "2020-10-14"],
                  ["2020-10-15", "2020-11-14"],
                  ["2020-11-15", "2020-12-14"],
                  ["2020-12-15", "2021-01-14"],
                  ["2021-01-15", "2021-02-14"],
                  ["2021-02-15", "2021-03-14"],
                  ["2021-03-15", "2021-04-14"],
                  ["2021-04-15", "2021-05-14"],
                  ["2021-05-15", "2021-06-14"],
                  ["2021-06-15", "2021-07-14"],
                  ["2021-07-15", "2021-08-14"]]

In [12]:
monthly_cases_list = []

for dis in set(modified_df_covid["District_Key"]):
    
    #Initiating month id  to 1
    time_id = 1
    #df store the dataframe for single district
    df = modified_df_covid[modified_df_covid["District_Key"] == dis]
    #Looping over all month from month_wise_date
    for i in range(len(month_wise_date)):
        
        start = month_wise_date[i][0]   #Starting date
        end = month_wise_date[i][1]       #ending date
        period = df.Date.between(start, end)   #Period = Ending date - starting data +1 
        
        #Summning over period to find the number of cases in month
        cases = np.sum(df[period]["Confirmed"])

        monthly_cases_list.append([dis, time_id, cases])
        #increasing timeid by 1
        time_id = time_id + 1

monthly_cases_df = pd.DataFrame(monthly_cases_list, columns=["districtid", "timeid", "cases"])
monthly_cases_df.sort_values(by = ["districtid", "timeid"], inplace=True)

### District Overall Cases

Storing overall number of cases for each district 

In [13]:
overall_cases_list = []

for dis in set(monthly_cases_df["districtid"]):
    time_id = 1
    df = monthly_cases_df[((monthly_cases_df["districtid"] == dis))]
    cases = sum(list(df["cases"]))
    overall_cases_list.append([dis,1, cases])
overall_cases_df = pd.DataFrame(overall_cases_list, columns=["districtid","timeid", "cases"])
overall_cases_df.sort_values(by = ["districtid"], inplace=True)

### Storing the final result for district

**weekly_cases_df** : District wise weekly cases
<br>
**monthly_cases_df**  : District monthly cases
<br>
**overall_cases_df**  : District wise Overall cases

In [14]:
weekly_cases_df.to_csv("Dataset/Output Files/cases-week.csv", index=False)
monthly_cases_df.to_csv("Dataset/Output Files/cases-month.csv", index=False)
overall_cases_df.to_csv("Dataset/Output Files/cases-overall.csv", index = False)

## Finding Weekly and Monthly cases for States and Country for Question 4

### Storing Cases state wise

**covid_state** will store datewise cases for every state

In [15]:
cases = []
for state in unique_state:
    df = modified_df_covid[modified_df_covid["State_Code"] == state]
    for date in date_range:
        case = np.sum(df[df["Date"] == date]["Confirmed"])
        cases.append([state, date, case])
        
covid_state = pd.DataFrame(cases, columns=["State_Code", "Date", "Confirmed"])
covid_state.sort_values(by = ["State_Code", "Date"], inplace = True)

### State wise weekly

In [16]:
state_weekly_cases_list = []
for state in unique_state:
    i = 0
    j = 4
    time_id = 1
    
    df = covid_state[covid_state["State_Code"] == state]
    while i < len(date_range):

        cases = np.sum(df.iloc[i : i + 7]["Confirmed"])
        week = "week " + str(time_id)
        state_weekly_cases_list.append([state, time_id, cases])
        
        time_id = time_id + 1
        
        i = i + 7
        
        cases = np.sum(df.iloc[j : j + 7]["Confirmed"])
        
        state_weekly_cases_list.append([state, time_id, cases])
        
        time_id = time_id + 1
        
        j = j + 7
        
        
state_weekly_cases_df = pd.DataFrame(state_weekly_cases_list, columns=["stateid", "timeid", "cases"])
state_weekly_cases_df.sort_values(by=["stateid", "timeid"], inplace=True)

### State wise monthly

In [17]:
state_monthly_cases_list = []
for state in unique_state:

    time_id = 1
 
    df = covid_state[covid_state["State_Code"] == state]

    for i in range(len(month_wise_date)):
        
        start = month_wise_date[i][0]
        end = month_wise_date[i][1]       
        period = df.Date.between(start, end)
        
        cases = np.sum(df[period]["Confirmed"])

        state_monthly_cases_list.append([state, time_id, cases])
        
        time_id = time_id + 1
state_monthly_cases_df = pd.DataFrame(state_monthly_cases_list, columns=["stateid", "timeid", "cases"])
state_monthly_cases_df.sort_values(by=["stateid", "timeid"], inplace=True)

### Storing cases at country level

In [18]:
cases = []
for date in date_range:
    case = np.sum(modified_df_covid[modified_df_covid["Date"] == date]["Confirmed"])
    cases.append([date,case])
df = pd.DataFrame(cases, columns=["Date", "Confirmed"])
df.sort_values(by = ["Date"], inplace=True)

### Weekly cases at country level

In [19]:
country_weekly_cases_list = []

i = 0
j = 4
time_id = 1


while i < len(date_range):

    cases = np.sum(df.iloc[i : i + 7]["Confirmed"])
        
    country_weekly_cases_list.append(["India", time_id, cases])
        
    time_id = time_id + 1
        
    i = i + 7
    
    cases = np.sum(df.iloc[j : j + 7]["Confirmed"])
        
    country_weekly_cases_list.append(["India", time_id, int(cases)])
        
    time_id = time_id + 1
        
    j = j + 7
    
counrty_weekly_cases_df = pd.DataFrame(country_weekly_cases_list, columns= ["counrty", "timeid", "cases"])
counrty_weekly_cases_df.sort_values(by = ["timeid"], inplace=True)

### Monthly Cases at Country Level

In [20]:
country_monthly_cases_list = []

time_id = 1
    
for i in range(len(month_wise_date)):
        
    start = month_wise_date[i][0]
    end = month_wise_date[i][1]       
    period = df.Date.between(start, end)
        
    cases = np.sum(df[period]["Confirmed"])

    country_monthly_cases_list.append(["India", time_id, cases])
        
    time_id = time_id + 1

counrty_monthly_cases_df = pd.DataFrame(country_monthly_cases_list, columns= ["counrty", "timeid", "cases"])
counrty_monthly_cases_df.sort_values(by = ["timeid"], inplace=True)

### Storing the results in data frame

In [21]:
weekly_cases_overlap_df.to_csv("Dataset/Additional Files/cases-week-overlaped-district.csv", index = False)

In [22]:
state_weekly_cases_df.to_csv("Dataset/Additional Files/cases-week-overlap-state.csv", index=False)
state_monthly_cases_df.to_csv("Dataset/Additional Files/cases-month-state.csv", index=False)

In [23]:
counrty_weekly_cases_df.to_csv("Dataset/Additional Files/cases-week-overlap-overall.csv", index=False)
counrty_monthly_cases_df.to_csv("Dataset/Additional Files/cases-month-overall.csv", index=False)